# Question 1

In [3]:
import pandas as pd
import numpy as np

In [178]:
#load Forecast Consumption sheet into df
df_forcasted_consumption = pd.read_excel(r'/Users/shirahcashriel/Downloads/gorilla_test/data.xlsx', sheet_name='Forecasted Consumption')
df_forcasted_consumption.head()

,Date,Meter ID,kWh
0,2020-10-01,1000000603,28.782474
1,2020-10-01,10588707,126.367711
2,2020-10-01,10626610,326.240595
3,2020-10-02,1000000603,26.690797
4,2020-10-02,10588707,118.322449


In [179]:
#load Meter List sheet into df
df_meters = pd.read_excel(r'/Users/shirahcashriel/Downloads/gorilla_test/data.xlsx', sheet_name='Meter List')
df_meters.set_index('Meter ID', inplace=True)
df_meters.head()

,Exit Zone,Annual Quantity (kWh)
Meter ID,,
1000000603,SC1,12337
10588707,EM2,75123
10626610,NW1,151779


In [180]:
#load Rates sheet into df
df_rates = pd.read_excel(r'/Users/shirahcashriel/Downloads/gorilla_test/data.xlsx', sheet_name='Rates')
df_rates.head()

,Date,Exit Zone,Annual Quantity (Min),Annual Quantity (Max),Rate (p/kWh)
0,2020-04-01,EA1,0,73200.0,0.0287
1,2020-04-01,EA2,0,73200.0,0.0287
2,2020-04-01,EA3,0,73200.0,0.0287
3,2020-04-01,EA4,0,73200.0,0.0287
4,2020-04-01,EM1,0,73200.0,0.0287


In [181]:
#replace NaN with infinity for max AQ
df_rates['Annual Quantity (Max)'].fillna(np.inf, inplace=True)
df_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1026 entries, 0 to 1025
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Date                   1026 non-null   datetime64[ns]
 1   Exit Zone              1026 non-null   object        
 2   Annual Quantity (Min)  1026 non-null   int64         
 3   Annual Quantity (Max)  1026 non-null   float64       
 4   Rate (p/kWh)           1026 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 40.2+ KB


In [182]:
date_ranges = df_rates['Date'].unique()
date_ranges

array(['2020-04-01T00:00:00.000000000', '2020-10-01T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-10-01T00:00:00.000000000',
       '2022-04-01T00:00:00.000000000', '2022-10-01T00:00:00.000000000',
       '2023-04-01T00:00:00.000000000', '2023-10-01T00:00:00.000000000',
       '2024-04-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [183]:
#bin date ranges to easily find appropriate daily rate 
df_forcasted_consumption['Date for Rate'] = pd.cut(df_forcasted_consumption['Date'], date_ranges, \
                                                  right=False, labels = [date for date in date_ranges[:-1]] )
df_forcasted_consumption.head()

,Date,Meter ID,kWh,Date for Rate
0,2020-10-01,1000000603,28.782474,2020-10-01
1,2020-10-01,10588707,126.367711,2020-10-01
2,2020-10-01,10626610,326.240595,2020-10-01
3,2020-10-02,1000000603,26.690797,2020-10-01
4,2020-10-02,10588707,118.322449,2020-10-01


In [184]:
#convert bin series from category to datetime
df_forcasted_consumption['Date for Rate'] = pd.to_datetime(df_forcasted_consumption['Date for Rate']).astype('datetime64[ns]')
df_forcasted_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2190 entries, 0 to 2189
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           2190 non-null   datetime64[ns]
 1   Meter ID       2190 non-null   int64         
 2   kWh            2190 non-null   float64       
 3   Date for Rate  2190 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1)
memory usage: 68.6 KB


In [185]:
#retrieve appropriate rate based on conditions of date, zone, and AQ range
def calc_rate(exit_zone, aq, date):
    rate = df_rates.loc[(df_rates['Date'] == date) &\
                    (df_rates['Exit Zone'] == exit_zone)&\
                    (df_rates['Annual Quantity (Min)'] <= aq) &\
                    (df_rates['Annual Quantity (Max)'] >= aq),['Rate (p/kWh)']]
    rate.reset_index(inplace=True)

    return rate.loc[0]['Rate (p/kWh)']
    
    

rate_for_day = []

for row in df_forcasted_consumption.T.loc[:]:
    meter = df_forcasted_consumption.loc[row]['Meter ID']
    exit_zone = df_meters.loc[meter]['Exit Zone']
    aq = df_meters.loc[meter]['Annual Quantity (kWh)']
    date = df_forcasted_consumption.loc[row]['Date for Rate']
    rate = calc_rate(exit_zone, aq, date)
    rate_for_day.append(rate)

df_forcasted_consumption['Rate for Day'] = rate_for_day

df_forcasted_consumption.head()


,Date,Meter ID,kWh,Date for Rate,Rate for Day
0,2020-10-01,1000000603,28.782474,2020-10-01,0.0333
1,2020-10-01,10588707,126.367711,2020-10-01,0.0228
2,2020-10-01,10626610,326.240595,2020-10-01,0.0285
3,2020-10-02,1000000603,26.690797,2020-10-01,0.0333
4,2020-10-02,10588707,118.322449,2020-10-01,0.0228


In [186]:
#calculate daily cost based on projected usage and daily rate, converted to £
df_forcasted_consumption['Cost for Day (£)'] = df_forcasted_consumption['kWh'] * df_forcasted_consumption['Rate for Day'] * 0.01 
df_forcasted_consumption.head()

,Date,Meter ID,kWh,Date for Rate,Rate for Day,Cost for Day (£)
0,2020-10-01,1000000603,28.782474,2020-10-01,0.0333,0.009585
1,2020-10-01,10588707,126.367711,2020-10-01,0.0228,0.028812
2,2020-10-01,10626610,326.240595,2020-10-01,0.0285,0.092979
3,2020-10-02,1000000603,26.690797,2020-10-01,0.0333,0.008888
4,2020-10-02,10588707,118.322449,2020-10-01,0.0228,0.026978


In [204]:
#sum total projected consumptions and cost by meter
df_distribution_charges = df_forcasted_consumption.groupby('Meter ID').sum().round(2).drop(columns=['Rate for Day'])
df_distribution_charges.rename(columns={'kWh':'Total Estimated Consumption (kWh)', 'Cost for Day (£)':'Total Cost (£)'}, inplace=True)
df_distribution_charges


,Total Estimated Consumption (kWh),Total Cost (£)
Meter ID,,
10588707,122450.0,28.50
10626610,303558.0,88.24
1000000603,24674.0,8.27


In [5]:
# Question 2
import random

df_rates = pd.read_excel(r'/Users/shirahcashriel/Downloads/gorilla_test/data.xlsx', sheet_name='Rates')
df_rates['Annual Quantity (Max)'].fillna(np.inf, inplace=True)
DATE_RANGES = df_rates['Date'].unique()
EXIT_ZONES = df_rates['Exit Zone'].unique()


def generate_list_meters(n):
    """generate a random list of n meters"""
    
    #generate list of meter ids
    meter_ids = np.random.randint(9999999,99999999, size=n)
    
    #assign exit zone
    exit_zones = random.choices(EXIT_ZONES, k=n)
    
    #generate AQ
    aqs = np.random.randint(5000, size=n)

    
    return list(zip(meter_ids, exit_zones, aqs))



# Question 3
def generate_consumption_table(meters, start_date, end_date):
    """generate random consumption usage for a given range for a list of meters"""
    
    dates_df = pd.DataFrame(data=[date for date in pd.date_range(start=start_date, end=end_date)],columns=['Date'])
    meters_df = pd.DataFrame([meter for meter in meters], columns=['Meter ID'])

    dates_df['key'] = 1
    meters_df['key'] = 1
    df = pd.merge(dates_df,meters_df,on='key').drop('key', 1)
    df['kWh'] = np.random.uniform(1,500,size=len(df))
    return df
        


# Question 4



def calc_rate(exit_zone, aq, date):
    rate = df_rates.loc[(df_rates['Date'] == date) &\
                    (df_rates['Exit Zone'] == exit_zone)&\
                    (df_rates['Annual Quantity (Min)'] <= aq) &\
                    (df_rates['Annual Quantity (Max)'] >= aq),['Rate (p/kWh)']]
    rate.reset_index(inplace=True)
    return rate.loc[0]['Rate (p/kWh)']
    





def generate_transportation_cost_table(meters_list,consumption_table):
    """Generate a transportation cost table for a given list of meters and their consumptions"""
    
    
    df_meters = pd.DataFrame(data=meters_list,columns=['Meter ID','Exit Zone','Annual Quantity (kWh)'])
    df_meters.set_index('Meter ID', inplace=True)
    
    
    consumption_table['Date for Rate'] = pd.cut(consumption_table['Date'], DATE_RANGES, \
                                                  right=False, labels = [d for d in DATE_RANGES[:-1]] )

    
    consumption_table['Date for Rate'] = pd.to_datetime(consumption_table['Date for Rate']).astype('datetime64[ns]')

    rate_for_day = []

    #create a list of daily rates given the conditions of exit zone, date, and aq
    for row in consumption_table.T.loc[:]:
        meter = consumption_table.loc[row]['Meter ID']
        exit_zone = df_meters.loc[meter]['Exit Zone']
        aq = df_meters.loc[meter]['Annual Quantity (kWh)']
        date = consumption_table.loc[row]['Date for Rate']
        
        
        rate = calc_rate(exit_zone, aq, date)
        rate_for_day.append(rate)

    consumption_table['Rate for Day'] = rate_for_day
    #calculate cost for daily usage in £ given the rate for that day and usage
    consumption_table['Cost for Day(£)'] = consumption_table['Rate for Day'] * consumption_table['kWh'] * 0.01
    
    #calculate total forecasted consumption and total cost in £
    transportation_cost_table = consumption_table.groupby('Meter ID').sum().round(2)
    transportation_cost_table.rename(columns={'kWh':'Total Estimated Consumption (kWh)', 'Cost for Day (£)':'Total Cost (£)'}, inplace=True)
    transportation_cost_table.drop(columns=['Rate for Day'], inplace=True)
    
    return transportation_cost_table


meters_list =  generate_list_meters(4)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('First')
%timeit generate_transportation_cost_table(meters_list, consumption_table)


meters_list =  generate_list_meters(4)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2024-01-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Second')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(15)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Third')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(15)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2024-01-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Fourth')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(35)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Fifth')
%timeit generate_transportation_cost_table(meters_list, consumption_table)


First
4.01 s ± 642 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Second
19.8 s ± 1.5 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Third
12.9 s ± 503 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Fourth
1min 16s ± 6.26 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
Fifth
36.7 s ± 4.56 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Comparing the different runs, we can see how poorly it scales, especially for longer periods. Comparing the fourth and fifth runs, we see that though the last run had more than twice as many meters, it took much less time because of how much shorter the time period was.

The recurring calls to calculate the rate for the day are unecessary and are time intensive - the rate is the same for the entire date range (the range used to determine which Rate (p/kWh) applies). A better way to calculate this would be to first reduce the dimensions by aggregating kWh usage per meter per date range.

In [13]:

import random

df_rates = pd.read_excel(r'/Users/shirahcashriel/Downloads/gorilla_test/data.xlsx', sheet_name='Rates')
df_rates['Annual Quantity (Max)'].fillna(np.inf, inplace=True)
DATE_RANGES = df_rates['Date'].unique()
EXIT_ZONES = df_rates['Exit Zone'].unique()


def generate_list_meters(n):
    """generate a random list of n meters"""
    
    #generate list of meter ids
    meter_ids = np.random.randint(9999999,99999999, size=n)
    
    #assign exit zone
    exit_zones = random.choices(EXIT_ZONES, k=n)
    
    #generate AQ
    aqs = np.random.randint(5000, size=n)

    
    return list(zip(meter_ids, exit_zones, aqs))


def generate_consumption_table(meters, start_date, end_date):
    """generate random consumption usage for a given range for a list of meters"""
    
    dates_df = pd.DataFrame(data=[date for date in pd.date_range(start=start_date, end=end_date)],columns=['Date'])
    meters_df = pd.DataFrame([meter for meter in meters], columns=['Meter ID'])

    dates_df['key'] = 1
    meters_df['key'] = 1
    df = pd.merge(dates_df,meters_df,on='key').drop('key', 1)
    df['kWh'] = np.random.uniform(1,500,size=len(df))
    return df
        


# Question 4 - improved



def calc_rate(exit_zone, aq, date):
    rate = df_rates.loc[(df_rates['Date'] == date) &\
                    (df_rates['Exit Zone'] == exit_zone)&\
                    (df_rates['Annual Quantity (Min)'] <= aq) &\
                    (df_rates['Annual Quantity (Max)'] >= aq),['Rate (p/kWh)']]
    rate.reset_index(inplace=True)
    return rate.loc[0]['Rate (p/kWh)']
    


def generate_transportation_cost_table(meters_list,consumption_table):
    """Generate a transportation cost table for a given list of meters and their consumptions"""
    
    
    df_meters = pd.DataFrame(data=meters_list,columns=['Meter ID','Exit Zone','Annual Quantity (kWh)'])
    df_meters.set_index('Meter ID', inplace=True)
    
    
    consumption_table['Date for Rate'] = pd.cut(consumption_table['Date'], DATE_RANGES, \
                                                  right=False, labels = [d for d in DATE_RANGES[:-1]] )

    
    consumption_table['Date for Rate'] = pd.to_datetime(consumption_table['Date for Rate']).astype('datetime64[ns]')
    
    aggregated_consumption_table = consumption_table.groupby(['Meter ID', 'Date for Rate']).sum().dropna()

    rate_for_period = []

    #create a list of daily rates given the conditions of exit zone, date, and aq
    for row in aggregated_consumption_table.T.loc[:]:
        meter = row[0]
        exit_zone = df_meters.loc[meter]['Exit Zone']
        aq = df_meters.loc[meter]['Annual Quantity (kWh)']
        date = row[1]
        
        
        rate = calc_rate(exit_zone, aq, date)
        rate_for_period.append(rate)

    aggregated_consumption_table['Rate for Period'] = rate_for_period
    #calculate cost for daily usage in £ given the rate for that day and usage
    aggregated_consumption_table['Cost for Period(£)'] = aggregated_consumption_table['Rate for Period'] * aggregated_consumption_table['kWh'] * 0.01
    
    #calculate total forecasted consumption and total cost in £
    transportation_cost_table = aggregated_consumption_table.groupby('Meter ID').sum().round(2)
    transportation_cost_table.rename(columns={'kWh':'Total Estimated Consumption (kWh)', 'Cost for Period (£)':'Total Cost (£)'}, inplace=True)
    transportation_cost_table.drop(columns=['Rate for Period'], inplace=True)
    
    return transportation_cost_table

meters_list =  generate_list_meters(4)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('First')
%timeit generate_transportation_cost_table(meters_list, consumption_table)


meters_list =  generate_list_meters(4)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2024-01-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Second')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(15)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Third')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(15)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2024-01-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Fourth')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

meters_list =  generate_list_meters(35)            
start_date = np.datetime64("2021-03-31")
end_date = np.datetime64("2021-10-05")
meters = [meters_list[i][0] for i in range(len(meters_list))]
consumption_table = generate_consumption_table(meters, start_date, end_date)
print('Fifth')
%timeit generate_transportation_cost_table(meters_list, consumption_table)

First
69.1 ms ± 24.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Second
133 ms ± 40.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Third
336 ms ± 55.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Fourth
411 ms ± 14.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Fifth
420 ms ± 68.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This improved code was undeniably faster and used significantly less resources. It clearly scales much better than the previous version.